<a href="https://colab.research.google.com/github/harbour22/darkcyan/blob/main/DarkCyan-yolo-trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## General Imports
from google.colab import drive # type: ignore
from IPython.display import clear_output # type: ignore
from pathlib import Path
import json
import shutil
import os
import time
from datetime import datetime


In [2]:
!pip install --upgrade git+https://github.com/harbour22/darkcyan.git
clear_output(wait=False)

In [3]:
!pip install ultralytics
clear_output(wait=False)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## App Imports
from darkcyan.darkcyan_training_utils import get_training_zip_name
from darkcyan.darkcyan_training_utils import save_config
from darkcyan.constants import GOOGLEDRIVE_DATA_ROOT, \
                               DEFAULT_GOOGLEDRIVE_YOLO_DATA_DIR, \
                               DEFAULT_GOOGLEDRIVE_YOLO_CONFIG_DIR, \
                               DEFAULT_GOOGLEDRIVE_YOLO_TRAINING_OUTPUT_DIR, \
                               DEFAULT_YOLO_TRAINING_CONFIG, \
                               DEFAULT_GOOGLEDRIVE_YOLO_ENGINE_DIR, \
                               YOLOMODELMAP, \
                               YoloBaseModels, \
                               DataType, DEFAULT_DET_TRAINING_YAML

from ultralytics import YOLO

Config file /root/.darkcyan/config.json does not exist, creating

Config key local_data_repository not found, attempting defaults

Config key scratch_dir not found, attempting defaults

Config key data_prefix not found, attempting defaults

Config key temp_dir not found, attempting defaults

In [ ]:
## Load config
config_file = Path(GOOGLEDRIVE_DATA_ROOT) / \
                   DEFAULT_GOOGLEDRIVE_YOLO_CONFIG_DIR / \
                   DEFAULT_YOLO_TRAINING_CONFIG

with open(config_file, "r", encoding="utf-8") as f:
    config = json.load(f)
config

{'version': '4.1',
 'type': 'det',
 'training_data': 'limetree_v4.1_det.zip',
 'basemodel': 'large',
 'epochs': 300,
 'imgsz': 640,
 'batchsize': 24,
 'config_creation_time': '2023-05-06 20:06:13',
 'created_by': 'chris'}

In [ ]:
data_type = DataType[config['type']]
model_size = YoloBaseModels[config['basemodel']]
batch = config['batchsize']
epochs = config['epochs']

project_path = Path(GOOGLEDRIVE_DATA_ROOT) / \
               DEFAULT_GOOGLEDRIVE_YOLO_TRAINING_OUTPUT_DIR / \
               config['type'] / f"darkcyan_{config['version']}" / \
               model_size.name

data = data_path = Path(f"/content/{config['type']}_training_data")
imgsz = config['imgsz']

if data_type == DataType.det:
  data = data / DEFAULT_DET_TRAINING_YAML

In [ ]:
zip_filename = Path(GOOGLEDRIVE_DATA_ROOT) / DEFAULT_GOOGLEDRIVE_YOLO_DATA_DIR / get_training_zip_name(config['version'], data_type, True)
shutil.unpack_archive(zip_filename,data_path)

In [ ]:
last_run = project_path / 'train' / 'weights' / 'last.pt'
if last_run.exists():
  base_model = last_run
  resume=True
else:
  base_model = YOLOMODELMAP[data_type][model_size]
  resume=False

print(base_model)

model = YOLO(base_model) # pass any model type
start_time = time.time()
model.train(epochs=300, resume=resume, project=project_path.as_posix(), batch=batch, data=data.as_posix(), imgsz=imgsz, exist_ok = True)
end_time = time.time()

In [ ]:
engine_file_name = f"yolov8_{config['version']}_{config['basemodel']}-{config['type']}.pt"
config_file_name = f"yolov8_{config['version']}_{config['basemodel']}-{config['type']}.json"
training_output = project_path / 'train' / 'weights' / 'best.pt'
engine_dir = Path(GOOGLEDRIVE_DATA_ROOT) / DEFAULT_GOOGLEDRIVE_YOLO_ENGINE_DIR
engine_output = engine_dir / engine_file_name
config_output = Path(GOOGLEDRIVE_DATA_ROOT) / DEFAULT_GOOGLEDRIVE_YOLO_ENGINE_DIR / config_file_name

if(not Path(engine_dir).exists()):
  engine_dir.mkdir(parents=True)

shutil.copy(training_output, engine_output)

config['output_engine'] = engine_file_name
config['colab_version'] = os.environ['COLAB_RELEASE_TAG']
config['elapsed_training_time_mins'] = f'{(end_time - start_time)/60:.2f}'
config["training_end_time"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


save_config(config, config_output)
